# Compare SID methods

In this notebook we will test the code to compare efficiency of System Identification by different metheods on the random data series generated by Lindblad master equation.

For each random quantum system in random initial state (determined by particular seed) we take following steps:
1. Generate time series of density matrices using Lindblad master eqution with random $H$, $J$ and initial density matrix $\rho_0$ given with particular seed.
\begin{align}
    \dot{\rho} = \mathcal{L}(\rho) = - \frac{i}{h} [H,\rho] + \sum_i \left ( J_i \rho J_i^\dagger - \frac{1}{2} J_i^\dagger J_i \rho - \frac{1}{2} \rho J_i^\dagger J_i \right )
\end{align}
2. Calculate symbolicaly objectives for polynomial optimization with (1) Pade and (2) Simpson methods assuming Lindblad evolution and perform optimization to find estimated $H$, $J$ and $\rho_0$.
3. Calculate symbolicaly objective asuming (3) Kraus evolution to find Kraus operators $K$ and initial state of the system.
\begin{aligned}
    \rho(t) & = \sum_{k=1}^{\ell} K_k^{} \rho(0) K_k^{\dagger}.
\end{aligned}
3. Perform (4) Linear system identification assuming system is LTI (Linear Time Independent) to find matrices $A$, $C$ and initial internal state of the system $x_0$:
\begin{align}
    \frac{d\mathbf{x}(t)} {dt} = A_c  \mathbf{x}(t)\\
    \mathbf{y}(t) = C \mathbf{x}(t)
\end{align}
4. Reconstruct time series of evolution with identified parameters of the system for all methods (1)-(4) and compare them with initial time series (considered as exact true evolution of the system).
5.Evaluate efficiency of methods by calculating minimum fidelity between exact and reconstructed time series.
\begin{align}
    F_{\min} & = \min_{i = 0, \ldots, N} F\left(\rho^{\rm exact}_{(i)}, \rho^{\rm sid}_{(i)} \right), \label{eq:fmin} \\
    F\left(\rho^{\rm exact}_{(i)}, \rho^{\rm sid}_{(i)} \right) & = \operatorname{Tr} \sqrt{ \sqrt{\rho^{\rm exact}_{(i)}} \rho^{\rm sid}_{(i)} \sqrt{\rho^{\rm exact}_{(i)}}}.
\end{align}

### Multi threading
Lets check number of threads available (https://docs.julialang.org/en/v1/manual/multi-threading/)

In [1]:
Threads.nthreads()

6

### Seeds
Lets generate unique set of seeds to generate random systems

In [2]:
# n_samples = 100000
n_samples = 1
seeds = rand(UInt, n_samples)
@assert allunique(seeds)

### Time span

In [ ]:
tₘₐₓ = 2.0 # maximum time 
Δt = 0.01     # time step
t = [0:Δt:tₘₐₓ;] # time span
time_steps = length(t)

### Library
We will use our own library of functions and a number of other packages listed in the external file.

In [3]:
include("LiPoSID.jl")

Main.LiPoSID

### Auxilary functions

In [23]:
using Dates
using QuantumOptics

function Lindblad_time_evolution(basis, ρ0, time_span, H_sid, J_sid)
         
    ρ0_sid = DenseOperator(basis, Hermitian(ρ0)) 

    H_sid = DenseOperator(basis, H_sid) # reconstructed Hamiltonian of the system
    J_sid = DenseOperator(basis, J_sid) # reconstracted Lindblad decipator
    
    time, ρ_sid_ser  = timeevolution.master(time_span, ρ0_sid, H_sid, [J_sid])
    
    ρ_sid = [ρₜ.data for ρₜ in ρ_sid_ser]

end

function Kraus_time_evolution(ρ0, time_steps, K_list)
    
    K1_sid, K2_sid = K_list
            
    ρ = LiPoSID.timeevolution_kraus(time_steps, ρ0, [K1_sid, K2_sid])

end

function min_fidelity_between_series(ρ1, ρ2)
    @assert length(ρ1) == length(ρ1) 
    minimum([abs(fidelity(ρ1, ρ2)) for i in 1:length(ρ1)])
end

min_fidelity_between_series (generic function with 1 method)

### Define symbolic operators
We need symbolic Hamiltonian, at least one Lindblad dissipator and two to build Krauss operators to build expressions for Polynonial minimization

For Lindbland dynamics
\begin{align}
    \dot{\rho} = \mathcal{L}(\rho) = - \frac{i}{h} [H,\rho] + \sum_i \left ( J_i \rho J_i^\dagger - \frac{1}{2} J_i^\dagger J_i \rho - \frac{1}{2} \rho J_i^\dagger J_i \right )
\end{align}

In [24]:
using DynamicPolynomials

@polyvar x[1:4]
H_symb = [ 1.0 * x[1]              x[3] + im * x[4]
           x[3] - im * x[4]        x[2]             ]

@polyvar a[1:2, 1:2]
@polyvar b[1:2, 1:2]
J_symb = 1.0 * a + im * b

2×2 Matrix{Polynomial{true, ComplexF64}}:
 a₁₋₁ + (0.0+1.0im)b₁₋₁  a₁₋₂ + (0.0+1.0im)b₁₋₂
 a₂₋₁ + (0.0+1.0im)b₂₋₁  a₂₋₂ + (0.0+1.0im)b₂₋₂

For Krauss dynamics
\begin{aligned}
    \rho(t) & = \sum_{k=1}^{\ell} K_k^{} \rho(0) K_k^{\dagger}.
\end{aligned}

In [25]:
using DynamicPolynomials

@polyvar a1[1:2, 1:2]
@polyvar b1[1:2, 1:2]
K1_symb = 1.0 * a1 + im * b1

@polyvar a2[1:2, 1:2]
@polyvar b2[1:2, 1:2]
K2_symb = 1.0 * a2 + im * b2

K_symb_list = [K1_symb, K2_symb]

2-element Vector{Matrix{Polynomial{true, ComplexF64}}}:
 [a1₁₋₁ + (0.0 + 1.0im)b1₁₋₁ a1₁₋₂ + (0.0 + 1.0im)b1₁₋₂; a1₂₋₁ + (0.0 + 1.0im)b1₂₋₁ a1₂₋₂ + (0.0 + 1.0im)b1₂₋₂]
 [a2₁₋₁ + (0.0 + 1.0im)b2₁₋₁ a2₁₋₂ + (0.0 + 1.0im)b2₁₋₂; a2₂₋₁ + (0.0 + 1.0im)b2₂₋₁ a2₂₋₂ + (0.0 + 1.0im)b2₂₋₂]

### Define basis for quantum system

In [ ]:
using QuantumOptics
basis = NLevelBasis(2)

### Main loop
Now we can assemble all tasks togeter inside the for loop to perform it in the multi-thread mode:

In [2]:
Threads.@threads for i=1:n_samples
    
    # Random time series of density matrices using Lindblad master eqution
    ρ, H_exact, J_exact = LiPoSID.rand_Linblad_w_noise(basis, seed[i], w, t)
    
    # Polynomial objectives with (1) Pade and (2) Simpsom methods
    obj_pade = LiPoSID.pade_obj(ρ, t, H_symb, J_symb)
    obj_simp = LiPoSID.simpson_obj(ρ, t, H_symb, J_symb)
    
    solution_pade = minimize_global(obj_pade)
    solution_simp = minimize_global(obj_simp)
    
    H_sid_pade = subs(H_symb, solution_pade)
    J_sid_pade = subs(J_symb, solution_pade)
    ρ_sid_pade = Lindblad_time_evolution(basis, solution_pade, ρ[1], t, H_sid_pade, J_sid_pade)
    
    H_sid_simp = subs(H_symb, solution_simp)
    J_sid_simp = subs(J_symb, solution_simp)  
    ρ_sid_simp = Lindblad_time_evolution(basis, solution_simp, ρ[1], t, H_sid_simp, J_sid_simp)
    
    fidelity_pade = min_fidelity_between_series(ρ_sid_pade, ρ)
    fidelity_simp = min_fidelity_between_series(ρ_sid_simp, ρ)     
    
    # Polynomial objective and constrain assuming (3) Kraus evolution
    obj_kraus, constr_kraus = LiPoSID.kraus(ρ, K_symb_list)
    solution_kraus = min2step(obj_kraus, constr_kraus)
    K1_sid = subs(K1_symb, solution_kraus)
    K2_sid = subs(K2_symb, solution_kraus)
    ρ_sid_kraus = LiPoSID.timeevolution_kraus(time_steps, ρ[1], [K1_sid, K2_sid])
    fidelity_kraus = min_fidelity_between_series(ρ_sid_kraus, ρ) 
    
    # (4) Linear SID as benckmark
    A, C, x0 = LiPoSID.lsid_ACx0(LiPoSID.bloch(ρ), Δt, δ = 1e-6)
    bloch_sid = LiPoSID.propagate(A, C, x0, time_steps)
    ρ_lsid = LiPoSID.rho_series_from_bloch(bloch_sid)
    fidelity_lsid = min_fidelity_between_series(ρ_lsid, ρ)
    
    
    # Save results to HDF5
    file_name = "LiPoSID_compare_methods_started_" * string(Dates.format(now(), "yyyy-u-dd_at_HH-MM")) * ".h5" 
    fid=h5open(file_name,"w")
    seed_group = g_create(fid,"seed")
    
    seed_group["H_exact"] = H_exact
    seed_group["J_exact"] = J_exact
    seed_group["rho0"] = ρ[1]
    
    
    seed_group["H_sid_pade"] = H_sid_pade
    seed_group["J_sid_pade"] = J_sid_pade
    
    seed_group["H_sid_simp"] = H_sid_simp
    seed_group["J_sid_simp"] = J_sid_simp
    
    seed_group["K1_sid"] = K1_sid
    seed_group["K2_sid"] = K2_sid
   
    seed_group["A"] = A
    seed_group["C"] = C  
    seed_group["x0"] = x0    
        
    seed_group["fidelity_pade"] = fidelity_pade
    seed_group["fidelity_simp"] = fidelity_simp
    seed_group["fidelity_kraus"] = fidelity_kraus
    seed_group["fidelity_lsid"] = fidelity_lsid

    close(fid)

end

LoadError: UndefVarError: n_samples not defined